In [1]:
import cadquery as cq
workplane = cq.Workplane("XY")

In [2]:
from box_generator.config import BoxConfig

import yaml

with open ("config.yml", "r") as file:
    config = yaml.safe_load(file)

box_config = BoxConfig.load(config)

In [68]:
USB_C_PORT = (
    cq.Workplane("XY")
    .rect(9.5, 3.5)
    .extrude(5)
    .edges("|Z")  # Select vertical edges
    .fillet(1.7)
    .rotate((0, 0, 0), (0, 1, 0), 90)
    .rotate((0, 0, 0), (1, 0, 0), 90)
)

MICRO_HDMI = (
    cq.Workplane("XY")
    .rect(6.5,3.4)
    .extrude(5)
    .rotate((0, 0, 0), (0, 1, 0), 90)
    .rotate((0, 0, 0), (1, 0, 0), 90)
)

In [70]:
class Component:
    width: float = 0
    height: float = 0
    depth: float = 0

    x: float = 0
    y: float = 0
    z: float = 0

    rotation: float = 0

    def __init__(self, box_config: BoxConfig):
        pass

class RaspberryPi5(Component):
    def __init__(self):
        self.width = 85
        self.height = 17.0
        self.depth = 56
        
        self._model = cq.importers.importStep("RP 5.stp").translate([self.width/2, self.depth/2, 0])
        # 1.6mm board thickens 3.2 mm for the usb-c port
        self._usb_c_cutout = USB_C_PORT.rotate((0,0,0),(0,0,1),-90).translate([11.2,0,1.6+3.2/2])
        self._hdmi_1_cutout = MICRO_HDMI.rotate((0,0,0),(0,0,1),-90).translate([25.8,0,1.6+3.4/2])
        self._hdmi_2_cutout = MICRO_HDMI.rotate((0,0,0),(0,0,1),-90).translate([39.2,0,1.6+3.4/2])

        self._visualization = cq.Workplane("XY")
        self._visualization = self._visualization.add(self._model)
        self._visualization = self._visualization.add(self._usb_c_cutout)
        self._visualization = self._visualization.add(self._hdmi_1_cutout)
        self._visualization = self._visualization.add(self._hdmi_2_cutout)
        
    
    def apply(self, box: cq.Workplane, box_config: BoxConfig) -> cq.Workplane:
        cuts = cq.Workplane("XY").box(1,2,1).translate([12,12,12])

        return box.union(cuts)


rpi = RaspberryPi5()

cq.exporters.export(rpi._visualization, "output.stl")

In [4]:
shell = cq.Workplane("XY").box(box_config.width, box_config.length, box_config.height)
inner_cut = (cq.Workplane("XY")
             .box(box_config.width - 2 * box_config.wall_thickness, box_config.length - 2 * box_config.wall_thickness, box_config.height)
             .translate([0,0,box_config.floor_thickness]))
box = shell.cut(inner_cut)

rpi = RaspberryPi5()

box = rpi.apply(box, box_config)
box = box.union(x)
display(box)


cq.exporters.export(box, 'output.stl')